In [214]:
"""
Создание частотных таблиц для каждой из колонок таблицы

"""

'\nСоздание частотных таблиц для каждой из колонок таблицы\n\n'

In [215]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
import time
from collections import Counter

In [216]:
name_file_data_groupby = 'data/Эксперты ВСР.xlsx'
path_to_end_folder_groupby = 'data'


In [217]:
df = pd.read_excel(name_file_data_groupby)

print(f'Колонки в таблице {df.columns}')

# Создаем шрифт которым будем выделять названия таблиц
font_name_table = Font(name='Arial Black', size=15, italic=True)
# Создаем файл excel
wb = openpyxl.Workbook()

Колонки в таблице Index(['ФИО', 'Пробелы',
       'Организация, город   Отметка о пролонгации свидетельства', 'Регион',
       '№ свидетельства', 'Компетенция', 'Дата выдачи', 'Срок действия (лет)',
       'Телефон', 'Имейл'],
      dtype='object')


In [218]:
# Проверяем наличие возможных дубликатов ,котороые могут получиться если обрезать по 30 символов
lst_length_column = [column[:30] for column in df.columns]

check_dupl_length = [k for k,v in Counter(lst_length_column).items() if v>1]


In [219]:
check_dupl_length

[]

In [220]:
# проверяем наличие объединенных ячеек
check_merge = [ column for column in df.columns if 'Unnamed' in column]
# если есть хоть один Unnamed то просто заменяем названия колонок на Колонка №цифра
if check_merge or check_dupl_length:
    df.columns = [f'Колонка №{i}' for i in range(1,df.shape[1]+1)]

In [221]:
# Делаем названия колонок строковыми
df.columns = list(map(str,list(df.columns)))
# Добавляем столбец для облегчения подсчета по категориям
df['Для подсчета'] = 1

In [222]:
# Создаем листы
for idx,name_column in enumerate(df.columns):
    # Делаем короткое название не более 30 символов
    wb.create_sheet(title=name_column[:30],index=idx)

In [223]:

for idx,name_column in enumerate(df.columns):
    group_df = df.groupby([name_column]).agg({'Для подсчета': 'sum'})
    group_df.columns = ['Количество']
    

        
    # Сортируем по убыванию
    group_df.sort_values(by=['Количество'],inplace=True,ascending=False)
    

    for r in dataframe_to_rows(group_df, index=True, header=True):
        if len(r) != 1:
            wb[name_column[:30]].append(r)
    wb[name_column[:30]].column_dimensions['A'].width = 50
    


In [224]:

# генерируем текущее время
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
# Удаляем листы 
del wb['Sheet']
del wb['Для подсчета']
# Сохраняем итоговый файл
wb.save(f'{path_to_end_folder_groupby}/Подсчет частоты значений для всех колонок таблицы от {current_time}.xlsx')